In [1]:
import time

from pysat.formula import CNF
from pysat.process import Processor
from pysat.solvers import Solver

In [2]:
cnf = CNF(from_file='cnfs/march-2023-03-13-16-13-22-044416.cnf')

In [3]:
default_kwargs = {
    'rounds': 1,
    'block': False,
    'cover': False,
    'condition': False,
    'decompose': False,
    'elim': False,
    'probe': False,
    'probehbr': False,
    'subsume': False,
    'vivify': False
}

In [4]:
def process_and_solve(cnf, **kwargs):
    print('Process kwargs:', kwargs)
    processor = Processor(bootstrap_with=cnf)
    begin = time.time()
    processed = processor.process(**kwargs)
    end = time.time()
    print('Process time:',  end - begin)
    print('Processed nv, nc:', processed.nv, len(processed.clauses))
    with Solver(name='g4', bootstrap_with=processed, use_timer=True) as solver:
        solver.solve()
        if solver.get_status() == True:
            print('SAT:', solver.get_model())
        elif solver.get_status() == False:
            print('UNSAT:', solver.get_core())
        print('Proof:', solver.get_proof())
        print('Processed solve time:', solver.time())
        print('Solver stats:', solver.accum_stats())
        return solver.get_status()


In [5]:
kwargs_all_off = {k: False if isinstance(v, bool) else v for k, v in default_kwargs.items()}
kwargs_all_on = {k: True if isinstance(v, bool) else v for k, v in default_kwargs.items()}


In [6]:
process_and_solve(cnf, **kwargs_all_off)

Process kwargs: {'rounds': 1, 'block': False, 'cover': False, 'condition': False, 'decompose': False, 'elim': False, 'probe': False, 'probehbr': False, 'subsume': False, 'vivify': False}
Process time: 13.703529119491577
Processed nv, nc: 1000 803350
UNSAT: None
Proof: None
Processed solve time: 0.045637329000001614
Solver stats: {'restarts': 1, 'conflicts': 26, 'decisions': 696, 'propagations': 753}


False

In [7]:
process_and_solve(cnf, **kwargs_all_on)

Process kwargs: {'rounds': 1, 'block': True, 'cover': True, 'condition': True, 'decompose': True, 'elim': True, 'probe': True, 'probehbr': True, 'subsume': True, 'vivify': True}
Process time: 12.075571537017822
Processed nv, nc: 1000 565466
UNSAT: None
Proof: None
Processed solve time: 0.0327479540000013
Solver stats: {'restarts': 1, 'conflicts': 11, 'decisions': 461, 'propagations': 478}


False

In [8]:
processor = Processor(bootstrap_with=cnf)
begin = time.time()
processed = processor.process(**kwargs_all_on)


In [9]:
# with Solver(bootstrap_with=processed, use_timer=True) as solver:
#     res = None
#     i = 0
#     while res is None:
#         if i % 100000000 == 0:
#             print(solver.accum_stats())
#         i += 1
#         solver.conf_budget(budget=1)
#         solver.prop_budget(budget=1)
#         res = solver.solve_limited()
#     print(solver.accum_stats())
    